In [1]:
!nvidia-smi

Sun Nov  2 17:11:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

connect google drive to google colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


check the content of drive whether connected or not

installing ultralytics

In [1]:
!pip install -U ultralytics opencv-python pillow matplotlib pandas albumentations

check th eno of data we have in drive

In [2]:
import os

root = "/content/drive/MyDrive/Crop_Weed_Project/dataset"
print("Images:", len(os.listdir(root + "/images")))
print("Labels:", len(os.listdir(root + "/labels")))


Images: 1300
Labels: 1300


create folder for train data, valid data and test data

In [14]:
import os

base_path = "/content/drive/MyDrive/Crop_Weed_Project/dataset_split"
os.makedirs(base_path + "/train/images", exist_ok=True)
os.makedirs(base_path + "/train/labels", exist_ok=True)
os.makedirs(base_path + "/valid/images", exist_ok=True)
os.makedirs(base_path + "/valid/labels", exist_ok=True)
os.makedirs(base_path + "/test/images", exist_ok=True)
os.makedirs(base_path + "/test/labels", exist_ok=True)

print("Train/Valid/Test folders created!")

Train/Valid/Test folders created!


this will split the data automatically

In [4]:
import os, random, shutil

src_images = "/content/drive/MyDrive/Crop_Weed_Project/dataset/images"
src_labels = "/content/drive/MyDrive/Crop_Weed_Project/dataset/labels"
dst = "/content/drive/MyDrive/Crop_Weed_Project/dataset_split"

files = [f for f in os.listdir(src_images) if f.lower().endswith(('.jpg','.png','.jpeg'))]
random.shuffle(files)

train_split = int(0.7 * len(files))
valid_split = int(0.9 * len(files))

for i, f in enumerate(files):
    if i < train_split:
        split = "train"
    elif i < valid_split:
        split = "valid"
    else:
        split = "test"

    img_src = os.path.join(src_images, f)
    lbl_src = os.path.join(src_labels, f.rsplit(".",1)[0] + ".txt")

    img_dst = os.path.join(dst, split, "images", f)
    lbl_dst = os.path.join(dst, split, "labels", f.rsplit(".",1)[0] + ".txt")

    shutil.copy(img_src, img_dst)
    shutil.copy(lbl_src, lbl_dst)

print("Dataset successfully splitted!")


Dataset successfully splitted!


count of splitted data in test, valid and train folder

In [7]:
def count(path):
    return len(os.listdir(path))

base = "/content/drive/MyDrive/Crop_Weed_Project/dataset_split"

print("Training Images:", count(base+"/train/images"))
print("Validation Images:", count(base+"/valid/images"))
print("Testing Images:", count(base+"/test/images"))

Training Images: 1191
Validation Images: 468
Testing Images: 241


create yaml file

In [4]:
data_yaml = """
train: /content/drive/MyDrive/Crop_Weed_Project/dataset_split/train/images
val: /content/drive/MyDrive/Crop_Weed_Project/dataset_split/valid/images
test: /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images

nc: 2
names: ['crop', 'weed']
"""

with open("/content/drive/MyDrive/Crop_Weed_Project/data.yaml", "w") as f:
    f.write(data_yaml)

print("data.yaml created successfully!")


data.yaml created successfully!


model training with pretrained model yolo8n.pt

In [3]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Pre-trained YOLO small model

results = model.train(
    data="/content/drive/MyDrive/Crop_Weed_Project/data.yaml",
    epochs=20,
    imgsz=512,
    batch=16,
    project="/content/drive/MyDrive/Crop_Weed_Project/",
    name="crop_weed_model",
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Crop_Weed_Project/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=F

stores teh result in the folder

In [7]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/Crop_Weed_Project/crop_weed_model2/weights/best.pt")

results = model.predict(
    source="/content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images",
    save=True
)

print("Prediction completed! Check the results folder.")



image 1/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1068.jpeg: 512x512 1 weed, 7.8ms
image 2/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1079.jpeg: 512x512 1 weed, 6.9ms
image 3/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1083.jpeg: 512x512 1 crop, 37.6ms
image 4/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1166.jpeg: 512x512 1 weed, 6.3ms
image 5/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1215.jpeg: 512x512 1 weed, 6.1ms
image 6/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1260.jpeg: 512x512 2 crops, 6.1ms
image 7/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1314.jpeg: 512x512 1 crop, 5.9ms
image 8/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1317.jpeg: 512x512 1 crop, 6.5ms
image 9/241 /content/drive/MyDrive/Crop_Weed_

shows actual result

In [8]:
import os
from google.colab.patches import cv2_imshow
import cv2

pred_path = "/content/runs/detect/predict2"

images = os.listdir(pred_path)

# Show first 5 images (you can change number)
for img_name in images[:5]:
    img_path = os.path.join(pred_path, img_name)
    img = cv2.imread(img_path)
    cv2_imshow(img)


Output hidden; open in https://colab.research.google.com to view.

In [10]:
# Replace with your correct path if folder name changed
!zip -r detection_results.zip /content/runs/detect/predict2


updating: content/runs/detect/predict2/ (stored 0%)
updating: content/runs/detect/predict2/agri_0_5568.jpg (deflated 5%)
updating: content/runs/detect/predict2/agri_0_156.jpg (deflated 4%)
updating: content/runs/detect/predict2/agri_0_3880.jpg (deflated 4%)
updating: content/runs/detect/predict2/agri_0_629.jpg (deflated 4%)
updating: content/runs/detect/predict2/agri_0_3690.jpg (deflated 5%)
updating: content/runs/detect/predict2/agri_0_4729.jpg (deflated 4%)
updating: content/runs/detect/predict2/agri_0_4516.jpg (deflated 5%)
updating: content/runs/detect/predict2/agri_0_9891.jpg (deflated 5%)
updating: content/runs/detect/predict2/agri_0_9843.jpg (deflated 6%)
updating: content/runs/detect/predict2/agri_0_8704.jpg (deflated 4%)
updating: content/runs/detect/predict2/agri_0_1166.jpg (deflated 6%)
updating: content/runs/detect/predict2/agri_0_491.jpg (deflated 5%)
updating: content/runs/detect/predict2/agri_0_2670.jpg (deflated 5%)
updating: content/runs/detect/predict2/agri_0_5182.jpg

result stored with prediction in zip file

In [13]:
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/Crop_Weed_Project/crop_weed_model2/weights/best.pt')

results = model.predict(
    source='/content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images',
    save=True,
    conf=0.25
)
print("Detection complete!")



image 1/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1068.jpeg: 512x512 1 weed, 6.2ms
image 2/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1079.jpeg: 512x512 1 weed, 5.7ms
image 3/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1083.jpeg: 512x512 1 crop, 20.0ms
image 4/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1166.jpeg: 512x512 1 weed, 6.2ms
image 5/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1215.jpeg: 512x512 1 weed, 6.2ms
image 6/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1260.jpeg: 512x512 2 crops, 5.9ms
image 7/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1314.jpeg: 512x512 1 crop, 5.9ms
image 8/241 /content/drive/MyDrive/Crop_Weed_Project/dataset_split/test/images/agri_0_1317.jpeg: 512x512 1 crop, 6.2ms
image 9/241 /content/drive/MyDrive/Crop_Weed_